In [44]:
import pandas as pd
import numpy as np

In [28]:
# Daysim formatted 2014 survey with adjusted weights
# trip = pd.read_csv(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt_daysim.csv')
trip = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt.xlsx')
parcel = pd.read_csv(r'L:\LODES\soundcast_2014_lodes\inputs\buffered_parcels.txt', sep=' ')

#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='opcl', right_on='parcelid')
#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='dpcl', right_on='parcelid', suffixes=['_o','_d'])

In [37]:
# Create standard modes likes SOV, HOV2+
trip['survey_mode'] = trip['mode'].copy()

# Walk
trip.ix[trip['mode'].isin([7]),'daysim_mode'] = 1

# Bike
trip.ix[trip['mode'].isin([6]),'daysim_mode'] = 2

# SOV
trip.ix[(trip['mode'].isin([1,4,4])) & (trip['travelers_total'] == 1),'daysim_mode'] = 3

# HOV2
trip.ix[(trip['mode'].isin([2,3,4,5])) & (trip['travelers_total'] == 2),'daysim_mode'] = 4

# HOV3+
trip.ix[(trip['mode'].isin([2,3,4,5])) & (trip['travelers_total'] > 2),'daysim_mode'] = 5

# Transit
# Drive Access
trip.ix[(trip['mode'].isin([8,9,10,11])) & (trip['mode_acc'].isin([3,4]) ),'daysim_mode'] = 6
# Walk/Bike Access
trip.ix[(trip['mode'].isin([8,9,10,11])) & (trip['mode_acc'].isin([1]) ),'daysim_mode'] = 7

# School Bus
trip.ix[trip['mode'].isin([12]),'daysim_mode'] = 8

# Other
trip.ix[trip['mode'].isin([17]),'daysim_mode'] = 9



In [40]:
# Calcualte 4k trip purposes

# Separate college student trips from regional survey trips
college_trips = trip[trip['hhid'] < 14000000]
trip = trip[trip['hhid'] >= 14000000]

# Home-Based Work (HBW) Trips, directly from home-to-work and work-to-home
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'] == 2), '4k_purp'] = 'HBW'
trip.ix[(trip['o_purpose'] == 2) & (trip['d_purpose'] == 1), '4k_purp'] = 'HBW'

# Home-Based shopping
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'].isin([4,5])), '4k_purp'] = 'Home-Based Shopping'
trip.ix[(trip['o_purpose'].isin([4,5])) & (trip['d_purpose'] == 1), '4k_purp'] = 'Home-Based Shopping'

# Home-Based School
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'] == 6), '4k_purp'] = 'School'
trip.ix[(trip['o_purpose'] == 6) & (trip['d_purpose'] == 1), '4k_purp'] = 'School'

# Home-Based Other
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'].isin([3,7,8,9,10,11,12,13,14,15,16])), '4k_purp'] = 'Home-Based Other'
trip.ix[(trip['o_purpose'].isin([3,7,8,9,10,11,12,13,14,15,16]) & (trip['d_purpose'] == 1)), '4k_purp'] = 'Home-Based Other'

# NHB Work-to-Other
trip.ix[(trip['o_purpose'] == 2) & (trip['d_purpose'] != 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'
trip.ix[(trip['o_purpose'] != 1) & (trip['d_purpose'] == 2) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'

# # NHB Other-to-Other (Destination and Origins are neither work nor home)
trip.ix[(trip['o_purpose'] != 2) & (trip['d_purpose'] != 1) & (trip['o_purpose'] != 1) 
        & (trip['d_purpose'] != 2) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB OtO'

In [55]:
# Add income cross classification

# Load household records to prepare data and generate cross-classification
hh = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx',
                  sheetname='Data')

# Reclassify household columns for cross-classification

# Number of Workers 0 -> 3+
hh['numworkers_crossclass'] = hh['numworkers']
hh.ix[hh['numworkers'] >= 3, 'numworkers_crossclass'] = 3
hh['numworkers_crossclass'] = hh['numworkers_crossclass'].astype('int')

# Household size
hh['hhsize_crossclass'] = hh['hhsize']
hh.ix[hh['hhsize'] >= 4, 'hhsize_crossclass'] = 4
hh['hhsize_crossclass'] = hh['hhsize_crossclass'].astype('int')

# Household income
# Note that exact ranges from 2014 do not match 2006
# old ranges: 30, 60, 90+
# new ranges: 35, 75, 100+
hh['income_crossclass'] = hh['hh_income_detailed_imp']
hh.ix[hh['hh_income_detailed_imp'] <= 3, 'income_crossclass'] = 1    # $35k  
hh.ix[(hh['hh_income_detailed_imp'] > 3) & (hh['hh_income_detailed_imp'] <= 5),    # $35-75k
           'income_crossclass'] = 2
hh.ix[(hh['hh_income_detailed_imp'] > 5) & (hh['hh_income_detailed_imp'] <= 6),    # $75-100k
           'income_crossclass'] = 3
hh.ix[hh['hh_income_detailed_imp'] >= 7, 'income_crossclass'] = 4    # >$100k

In [62]:
# Join the household information (and new cross-classes back to the trip records)
trip = pd.merge(trip,hh,how='left',on='hhid')

In [63]:
# Calcualte the mixed use variable on parcel data
# using daysim definition from Log2 functino
parcel['tot_hhemp_1'] = parcel['emptot_1'] + parcel['hh_1']
parcel['tot_hhemp_2'] = parcel['emptot_2'] + parcel['hh_2']

parcel['mixed_use_1'] = -1*(((parcel['emptot_1']/parcel['tot_hhemp_1'])*np.log(parcel['emptot_1']/parcel['tot_hhemp_1']) \
+ (parcel['hh_1']/parcel['tot_hhemp_1'])*np.log(parcel['hh_1']/parcel['tot_hhemp_1']))/np.log(2))

parcel['mixed_use_2'] = -1*(((parcel['emptot_2']/parcel['tot_hhemp_2'])*np.log(parcel['emptot_2']/parcel['tot_hhemp_2']) \
+ (parcel['hh_2']/parcel['tot_hhemp_2'])*np.log(parcel['hh_2']/parcel['tot_hhemp_2']))/np.log(2))

In [4]:
# Sum employment variables by TAZ

# Take a weighted average by emp, pop, and emp+pop for other variables


In [64]:
# Calculate TAZ totals for households, employment
totals_by_taz = parcel.groupby('taz_p').sum()[[
       'hh_p', u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'empedu_p',
       u'empfoo_p', u'empgov_p', u'empind_p', u'empmed_p', u'empofc_p',
       u'empret_p', u'empsvc_p', u'empoth_p', u'emptot_p']]
totals_by_taz.columns = [i.split('_p')[0]+'_taz_tot' for i in totals_by_taz.columns]
totals_by_taz = totals_by_taz.reset_index()


In [54]:
# Calculate a mixed use variable based on daysim calculation


In [65]:
# Calculate a weighted average for each TAZ
# based on the parcel buffers
# This avoids bias based on TAZ boundaries

# Weighted by parcel population, parcel employment, and combination of both
# For 1st buffer (0.25 miles?)

wt_fields = ['stugrd_',
       u'stuhgh_', u'stuuni_', u'empedu_', u'empfoo_', u'empgov_',
       u'empind_', u'empmed_', u'empofc_', u'empret_', u'empsvc_',
       u'empoth_', u'emptot_', u'parkdy_', u'parkhr_', u'ppricdy',
       u'pprichr','tstops_']

transit_fields = ['dist_lbus', u'dist_ebus', u'dist_crt', u'dist_fry', u'dist_lrt', u'dist_park']

for field in wt_fields:
    
    # Get numerator of weights (values within buffered areas)
    # to be divided by denominator of total hh/emp/hh+emp
    
    # Weight by parcel population, for 0.25 and 0.5 mile buffers
    parcel[field+'hh_wt1_tot'] = parcel[field+'1']*parcel['hh_p']    # .25 mile
    parcel[field+'hh_wt2_tot'] = parcel[field+'2']*parcel['hh_p']    # .5 mile
    
    # Weight by total parcel employment
    parcel[field+'emp_wt1_tot'] = parcel[field+'1']*parcel['emptot_p']    # .25 mile
    parcel[field+'emp_wt2_tot'] = parcel[field+'2']*parcel['emptot_p']    # .5 mile
    
    # Weight by households + employment
    parcel[field+'hhemp_wt1_tot'] = parcel[field+'1']*(parcel['emptot_p']+parcel['hh_p'])# .25 mile
    parcel[field+'hhemp_wt2_tot'] = parcel[field+'2']*(parcel['emptot_p']+parcel['hh_p'])# .5 mile

for field in transit_fields:
    
    parcel[field+'_hh_wt_tot'] = parcel[field]*parcel['hh_p']
    parcel[field+'_emp_wt_tot'] = parcel[field]*parcel['emptot_p']
    parcel[field+'_hhemp_wt_tot'] = parcel[field]*(parcel['emptot_p']+parcel['hh_p'])

In [66]:
# Compute weighted average for TAZ based on 

parcel['emptot_plus_hh'] = parcel['emptot_p']+parcel['hh_p']

for field in wt_fields:
    print field
    # Weights based on household size
    totals_by_taz[field+'hh_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt1_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'hh_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt2_tot']/parcel.groupby('taz_p').sum()['hh_p']
    
    # Weights based on employment
    totals_by_taz[field+'emp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt1_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    totals_by_taz[field+'emp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt2_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    
    # weights based on housheholds + employment
    totals_by_taz[field+'hhemp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt1_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']
    totals_by_taz[field+'hhemp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt2_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']
    
for field in transit_fields:
    totals_by_taz[field+'_hh_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_hh_wt_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'_emp_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_emp_wt_tot']/parcel.groupby('taz_p').sum()['emptot_p']
    totals_by_taz[field+'_hhemp_wt_avg'] = parcel.groupby('taz_p').sum()[field+'_hhemp_wt_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']

stugrd_
stuhgh_
stuuni_
empedu_
empfoo_
empgov_
empind_
empmed_
empofc_
empret_
empsvc_
empoth_
emptot_
parkdy_
parkhr_
ppricdy
pprichr
tstops_


In [67]:
# Fill NaNs with zeros - for zones with 0 hh or emp, divide by 0 results in NaN
totals_by_taz = totals_by_taz.fillna(0)

# Join totals by taz to the trip file

In [68]:
# Exclude trips to or from external zones
trip = trip[trip['o_taz10'] != 0]
trip = trip[trip['d_taz10'] != 0]

In [69]:
df = pd.merge(trip, totals_by_taz, left_on='o_taz10', right_on='taz_p', how='left')
df = pd.merge(df, totals_by_taz, left_on='d_taz10', right_on='taz_p', how='left', suffixes=['_o','_d'])
df = df.fillna(0)

In [71]:
df.to_csv(r'R:\4K\2014\Trip Generation\Trip Rates\2014-new-adjusted\trip_2014_adjusted_lu_vars.csv', 
          index=False, encoding='utf-8')

In [72]:
df

,personID,recordID,tripID,hhid,pernum,tripnum,total_trip,last,place_start,place_end,...,dist_crt_hhemp_wt_avg_d,dist_fry_hh_wt_avg_d,dist_fry_emp_wt_avg_d,dist_fry_hhemp_wt_avg_d,dist_lrt_hh_wt_avg_d,dist_lrt_emp_wt_avg_d,dist_lrt_hhemp_wt_avg_d,dist_park_hh_wt_avg_d,dist_park_emp_wt_avg_d,dist_park_hhemp_wt_avg_d
0,1410000101,A00001,141000010101,14100001,1,1,4,0.0,HOME,LAKE UNION CREW ROWING CLUB,...,3.202106,1.452049,1.493047,1.460707,1.607387,1.685237,1.623828,0.876387,0.812164,0.862824
1,1410000101,A00002,141000010102,14100001,1,2,4,0.0,LAKE UNION CREW ROWING CLUB,WORK,...,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,0.725991,0.687743,0.713793
2,1410000101,A00003,141000010103,14100001,1,3,4,0.0,WORK,QFC,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
3,1410000101,A00004,141000010104,14100001,1,4,4,1.0,QFC,HOME,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
4,1410000201,A00005,141000020101,14100002,1,1,12,0.0,HOME,WORK,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
5,1410000201,A00006,141000020102,14100002,1,2,12,0.0,WORK,HOME,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
6,1410000201,A00007,141000020103,14100002,1,3,12,0.0,HOME,WORK,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
7,1410000201,A00008,141000020104,14100002,1,4,12,0.0,WORK,HOME,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
8,1410000201,A00009,141000020105,14100002,1,5,12,0.0,HOME,BLUE SAUCER CAFE,...,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,0.705741,0.595702,0.691007
9,1410000201,A00010,141000020106,14100002,1,6,12,0.0,BLUE SAUCER CAFE,HOME,...,2.603836,1.148246,1.154609,1.149338,0.372105,0.374302,0.372482,0.212867,0.209362,0.212266
